In [1]:
# Code cited from https://github.com/mfurkanatac/MBTI-Detector, which had a very very similar problem to ours

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('train_cleaned.csv')

# nan values are present in the dataset
df['cleaned-text'].replace('', np.nan, inplace=True)
df.dropna(subset=['cleaned-text'], inplace=True)
df.dropna(subset=['label'], inplace=True)
df['label'] = df['label'].astype(int) # since label is float

# Split the data into training and validation datasets
train_text, temp_text, train_labels, temp_labels = train_test_split(df['cleaned-text'], df['label'], 
                                                                    random_state=2018, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label'])

# We'll further split the validation set into validation and test datasets
val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=2018, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [2]:
from transformers import BertTokenizerFast
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch

# Load pre-trained model tokenizer 
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Encode the text into tokens, masks, and segment flags
train_encodings = tokenizer(train_text.tolist(), truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_text.tolist(), truncation=True, padding=True, max_length=256)
test_encodings = tokenizer(test_text.tolist(), truncation=True, padding=True, max_length=256)

# Convert inputs to PyTorch tensors
train_inputs = torch.tensor(train_encodings['input_ids'])
validation_inputs = torch.tensor(val_encodings['input_ids'])
train_labels = torch.tensor(train_labels.tolist())
validation_labels = torch.tensor(val_labels.tolist())
train_masks = torch.tensor(train_encodings['attention_mask'])
validation_masks = torch.tensor(val_encodings['attention_mask'])
test_inputs = torch.tensor(test_encodings['input_ids'])
test_labels = torch.tensor(test_labels.tolist())
test_masks = torch.tensor(test_encodings['attention_mask'])

# Create DataLoader
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=16)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=16)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=16)

In [3]:
from transformers import BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import numpy as np
from tqdm import tqdm, trange

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 2) # spam (0 and 1) two labels
model.to(device)

# BERT fine-tuning parameters
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader)*3) # 3 epochs

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# Store our loss and accuracy for plotting
train_loss_set = []

cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Furkan\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [4]:
# Number of training epochs (authors recommend between 2 and 4)
epochs = 6

for _ in trange(epochs, desc="Epoch"):  
    # Training
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        if step % 250 == 0:
            print("Batch: {0}/{1}, Loss: {2:.2f}".format(step, len(train_dataloader), loss.item()))

    avg_train_loss = total_loss / len(train_dataloader)            
    train_loss_set.append(avg_train_loss)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

    # Validation
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    torch.save(model.state_dict(), f'checkpoint2_{_}.pt') # Save the model checkpoint

# Save the final model
torch.save(model.state_dict(), 'bert_model2.pt')



Epoch:   0%|          | 0/6 [00:00<?, ?it/s]

Batch: 0/1585, Loss: 0.62
Batch: 250/1585, Loss: 0.20
Batch: 500/1585, Loss: 0.54
Batch: 750/1585, Loss: 0.01
Batch: 1000/1585, Loss: 0.01
Batch: 1250/1585, Loss: 0.00
Batch: 1500/1585, Loss: 0.01
Average training loss: 0.17
Validation Accuracy: 0.96


Epoch:  17%|█▋        | 1/6 [13:30<1:07:32, 810.51s/it]

Batch: 0/1585, Loss: 0.02
Batch: 250/1585, Loss: 0.00
Batch: 500/1585, Loss: 0.00
Batch: 750/1585, Loss: 0.00
Batch: 1000/1585, Loss: 0.01
Batch: 1250/1585, Loss: 0.27
Batch: 1500/1585, Loss: 0.25
Average training loss: 0.06
Validation Accuracy: 0.97


Epoch:  33%|███▎      | 2/6 [26:58<53:57, 809.32s/it]  

...

Epoch:  67%|██████▋   | 4/6 [54:02<27:02, 811.18s/it]

Batch: 0/1585, Loss: 0.00
Batch: 250/1585, Loss: 0.00
Batch: 500/1585, Loss: 0.00
Batch: 750/1585, Loss: 0.00
Batch: 1000/1585, Loss: 0.00
Batch: 1250/1585, Loss: 0.00
Batch: 1500/1585, Loss: 0.00
Average training loss: 0.02
Validation Accuracy: 0.97


Epoch:  83%|████████▎ | 5/6 [1:07:37<13:32, 812.67s/it]

Batch: 0/1585, Loss: 0.00
Batch: 250/1585, Loss: 0.05
Batch: 500/1585, Loss: 0.28
Batch: 750/1585, Loss: 0.00
Batch: 1000/1585, Loss: 0.00
Batch: 1250/1585, Loss: 0.01
Batch: 1500/1585, Loss: 0.00
Average training loss: 0.03
Validation Accuracy: 0.97


Epoch: 100%|██████████| 6/6 [1:21:13<00:00, 812.22s/it]


In [5]:
def evaluate_test_set(test_dataloader):
    # Put model in evaluation mode to evaluate loss on the test set
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in test_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
        logits = outputs.logits

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Test Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))

# Call the function to evaluate the test set
evaluate_test_set(test_dataloader)

Test Accuracy: 0.9675
